In [ ]:
import boto3 
import botocore 
import pandas as pd 
from sagemaker import get_execution_role 
import numpy as np
import datetime
import math
from io import StringIO


role = get_execution_role() 

bucket = 'aws-data-exchange-mitchrad99' 
user_data_key = 'UserDatabase.csv' 
location_data_key = 'ce7d65bd-39cc-4cd3-838e-13aa964805dc.csv'
user_data_location = 's3://{}/{}'.format(bucket, user_data_key) 
location_data_location = 's3://{}/{}'.format(bucket, location_data_key) 

pd.read_csv(location_data_location) 
data = pd.read_csv(location_data_location, parse_dates=['datetime'])
data = data[data['country_name']=='United States']

#NE Data
#data = data.loc[(data['latitude'] < 42.7) & (data['latitude'] > 38) & (data['longitude'] < -70) & (data['longitude'] > -78)]

#NYC Data
data = data.loc[(data['latitude'] < 41) & (data['latitude'] > 40.5) & (data['longitude'] < -73.5) & (data['longitude'] > -75)]

geohash_precision = 5 #higher number => more precise...

data['geo_hash'] = data['geo_hash'].str[:geohash_precision]

user_input = pd.read_csv(user_data_location, parse_dates=['DiagnosisDateTime (S)', 'SymptomsDateTime (S)'])
user_input.columns = ['ID', 'Contact', 'DiagnosisDateTime', 'Infected', 'Symptom1', 'Symptom2', 'Symptom3', 'Symptom4', 'Symptom5', 'Symptom6', 'Symptom7', 'Symptom8', 'Symptom9', 'SymptomsDateTime']
user_input['Now']= datetime.datetime.now()

In [ ]:
ids = pd.DataFrame(data.device_id.unique(), columns=['id'])
ids['infected'] = False
ids['suspected'] = False
ids['last_infected'] = np.nan
ids['last_suspected'] = np.nan
ids['last_tested'] = np.nan
ids['problem_location_lats'] = np.empty((len(ids), 0)).tolist()
ids['problem_location_longs'] = np.empty((len(ids), 0)).tolist()
ids['symptom_score'] = 0.0
ids['lifestyle_score'] = 0.0
ids['total_score'] = 0.0

In [ ]:
locations = data.groupby('geo_hash').first().reset_index()
locations = locations[['geo_hash','latitude','longitude']]

In [ ]:
locations['contamination_score'] = 0.0
locations['diagnosed'] = np.empty((len(locations), 0)).tolist()
locations['diagnosed_times'] = np.empty((len(locations), 0)).tolist()
locations['suspected'] = np.empty((len(locations), 0)).tolist()
locations['suspected_times'] = np.empty((len(locations), 0)).tolist()
locations

In [71]:
for index, row in user_input.iterrows():
    id_index = ids.index[ids['id'] == row['ID']]
    if row['Infected'] == 'yes':
        ids.loc[id_index,'infected'] = True
        ids.loc[id_index,'total_score'] = 1.0
        ids.loc[id_index,'last_infected'] = row['DiagnosisDateTime']
    
        user_locations = data[(data['device_id'] == row['ID']) & (data['datetime'] >= row['DiagnosisDateTime'])]
        for location_index, location_row in user_locations.iterrows():
            if location_row['device_id'] not in locations.loc[locations['geo_hash'] == location_row['geo_hash'],'diagnosed'].iloc[0]:
                locations.loc[locations['geo_hash'] == location_row['geo_hash'],'diagnosed'].iloc[0] = locations.loc[locations['geo_hash'] == location_row['geo_hash'],'diagnosed'].iloc[0].append(location_row['device_id'])
                locations.loc[locations['geo_hash'] == location_row['geo_hash'],'diagnosed_times'].iloc[0] = locations.loc[locations['geo_hash'] == location_row['geo_hash'],'diagnosed_times'].iloc[0].append([location_row['device_id'], location_row['datetime'], location_row['datetime']])
            else:
                id_index = locations.loc[locations['geo_hash'] == location_row['geo_hash'],'diagnosed'].iloc[0].index(location_row['device_id'])
                if location_row['datetime'] < locations.loc[locations['geo_hash'] == location_row['geo_hash'],'diagnosed_times'].iloc[0][id_index][1]:
                    locations.loc[locations['geo_hash'] == location_row['geo_hash'],'diagnosed_times'].iloc[0][id_index][1] = location_row['datetime']
                if location_row['datetime'] > locations.loc[locations['geo_hash'] == location_row['geo_hash'],'diagnosed_times'].iloc[0][id_index][2]:
                    locations.loc[locations['geo_hash'] == location_row['geo_hash'],'diagnosed_times'].iloc[0][id_index][2] = location_row['datetime']
    else:
        ids.loc[id_index,'infected'] = False    
        ids.loc[id_index,'symptom_score'] = (row['Symptom1']*.05+row['Symptom2']*.05+row['Symptom3']*.05+row['Symptom4']*.05+row['Symptom5']*.05+row['Symptom6']*.05+row['Symptom7']*.05+row['Symptom8']*.05+row['Symptom9']*.05+row['Contact']*.5)
        individual_data = data.loc[data['device_id'] == row['ID']]
        
        
        individual_location_visit_times = (individual_data.assign(Data_Value=individual_data['datetime']).groupby(['geo_hash'])['Data_Value'].agg([('Min' , 'min'), ('Max', 'max')]).add_prefix('Day'))
        individual_location_visit_times.reset_index(level=0, inplace=True)
        individual_location_visit_times['infected_overlap'] = 0.0
        individual_location_visit_times['suspected_overlap'] = 0.0
        individual_location_visit_times['general_location_score'] = 0.0
        individual_location_visit_times['percent_infected'] = 0.0
        individual_location_visit_times['percent_suspected'] = 0.0
        individual_location_count = len(individual_location_visit_times)
        for index, row in individual_location_visit_times.iterrows():
            individual_location_visit_times.loc[index, 'general_location_score'] = locations.loc[locations['geo_hash'] == row['geo_hash']]['contamination_score'].iloc[0]    

            for individual in locations.loc[locations['geo_hash']==row['geo_hash'], 'diagnosed_times'].iloc[0]:
                person = individual[0]
                earliest = individual[1]
                latest = individual[2]
                latest_start = max(row['DayMin'], earliest)
                earliest_end = min(row['DayMax'], latest)
                delta = (earliest_end - latest_start).total_seconds() + 1
                overlap = max(0, delta)
                individual_location_visit_times.loc[index, 'infected_overlap'] += overlap
            for individual in locations.loc[locations['geo_hash']==row['geo_hash'], 'suspected_times'].iloc[0]:
                person = individual[0]
                earliest = individual[1]
                latest = individual[2]
                latest_start = max(row['DayMin'], earliest)
                earliest_end = min(row['DayMax'], latest)
                delta = (earliest_end - latest_start).total_seconds() + 1
                overlap = max(0, delta)
                individual_location_visit_times.loc[index, 'suspected_overlap'] += overlap
            
            individual_location_visit_times.loc[index, 'percent_infected'] = abs(individual_location_visit_times.loc[index, 'infected_overlap']/((individual_location_visit_times.loc[index, 'DayMin']-individual_location_visit_times.loc[index, 'DayMax']).total_seconds()+1)*(((individual_location_visit_times.loc[index, 'DayMin']-individual_location_visit_times.loc[index, 'DayMax']).total_seconds()+1)/60/60/24))
            individual_location_visit_times.loc[index, 'percent_suspected'] = abs(individual_location_visit_times.loc[index, 'suspected_overlap']/((individual_location_visit_times.loc[index, 'DayMin']-individual_location_visit_times.loc[index, 'DayMax']).total_seconds()+1)*(((individual_location_visit_times.loc[index, 'DayMin']-individual_location_visit_times.loc[index, 'DayMax']).total_seconds()+1)/60/60/24))

            if (individual_location_visit_times.loc[index, 'percent_infected'].iloc[0]>.05):
                ids.loc[ids['id'] == row['ID'], 'problem_location_lats'].iloc[0] = ids.loc[ids['id'] == row['ID'], 'problem_location_lats'].iloc[0].append(locations.loc[locations['geo_hash'] == row['geo_hash'], 'latitude'])
            if (individual_location_visit_times.loc[index, 'percent_suspected'].iloc[0]>.1):
                ids.loc[ids['id'] == row['ID'], 'problem_location_longs'].iloc[0] = ids.loc[ids['id'] == row['ID'], 'problem_location_longs'].iloc[0].append(locations.loc[locations['geo_hash'] == row['geo_hash'], 'longitude'])

            ids.loc[id_index,'lifestyle_score'] = math.sqrt(individual_location_count)/100 + individual_location_visit_times.loc[index, 'percent_infected']*.01 + individual_location_visit_times.loc[index, 'percent_infected']*.005
            ids.loc[id_index,'total_score'] = ids.loc[id_index,'symptom_score']*.75 + ids.loc[id_index,'lifestyle_score']*.25
            
        if ids.loc[id_index,'total_score'].iloc[0]>.5:
            user_locations = data[(data['device_id'] == row['ID']) & (data['datetime'] >= row['Now']-timedelta(days=2))]
            for location_index, location_row in user_locations.iterrows():
                if location_row['device_id'] not in locations.loc[locations['geo_hash'] == location_row['geo_hash'],'suspected'].iloc[0]:
                    locations.loc[locations['geo_hash'] == location_row['geo_hash'],'suspected'].iloc[0] = locations.loc[locations['geo_hash'] == location_row['geo_hash'],'suspected'].iloc[0].append(location_row['device_id'])
                    locations.loc[locations['geo_hash'] == location_row['geo_hash'],'suspected_times'].iloc[0] = locations.loc[locations['geo_hash'] == location_row['geo_hash'],'suspected_times'].iloc[0].append([location_row['device_id'], location_row['datetime'], location_row['datetime']])
                else:
                    id_index = locations.loc[locations['geo_hash'] == location_row['geo_hash'],'suspected'].iloc[0].index(location_row['device_id'])
                    if location_row['datetime'] < locations.loc[locations['geo_hash'] == location_row['geo_hash'],'suspected_times'].iloc[0][id_index][1]:
                        locations.loc[locations['geo_hash'] == location_row['geo_hash'],'suspected_times'].iloc[0][id_index][1] = location_row['datetime']
                    if location_row['datetime'] > locations.loc[locations['geo_hash'] == location_row['geo_hash'],'suspected_times'].iloc[0][id_index][2]:
                        locations.loc[locations['geo_hash'] == location_row['geo_hash'],'suspected_times'].iloc[0][id_index][2] = location_row['datetime']

In [50]:
ids['lat'] = 0.0
ids['long']= 0.0
for index, row in ids.iterrows():
    ids.loc[index, 'lat'] = locations[locations['geo_hash'] == row['']]

,device_id,datetime,latitude,longitude,ip_address,country_name,geo_hash
3,kzjvN7xm1r7RFfZobPFQIEyG5Fc=,2020-08-07 20:15:53,40.783675,-73.590224,172.58.227.251,United States,dr78j
9,+FtyYYYyzucn1AWkVZIGPbB1dN4=,2020-08-09 11:16:34,40.883161,-73.852214,107.242.117.3,United States,dr72x
38,ZBCEzJvsoq5+2ivMvpN0doGnc2U=,2020-08-07 17:05:37,40.749075,-73.816737,107.77.223.53,United States,dr5xb
59,Eddy5Zk5VYNz/kBMhnusSfpgfng=,2020-08-09 10:57:35,40.835600,-73.861200,72.80.53.93,United States,dr72r
61,QPoYhjCTpRq+XuZ0d0xkMukCX58=,2020-08-08 16:20:56,40.891139,-74.051792,99.203.54.209,United States,dr72d
...,...,...,...,...,...,...,...
44758,+FtyYYYyzucn1AWkVZIGPbB1dN4=,2020-08-08 21:07:33,40.883150,-73.852230,107.242.117.44,United States,dr72x
44771,grDrzrASYBwQT9+DlWMujLukJWM=,2020-08-07 22:12:48,40.676332,-74.005313,99.203.54.249,United States,dr5r7
44780,h7MhaMUIWiftTX+BaMn7DRUy4Y8=,2020-08-07 19:16:18,40.702700,-73.797240,172.58.230.163,United States,dr5x8
44785,QPoYhjCTpRq+XuZ0d0xkMukCX58=,2020-08-07 20:27:17,40.891771,-74.054063,99.203.145.219,United States,dr72d


In [73]:
bucket = 'aws-data-exchange-mitchrad99' 
csv_buffer = StringIO()
ids.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'ids.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'E6570B2A04B54027',
  'HostId': 'Ot2r9b5l8X63IhpqRLc+J2hvw7AAYItvYKD1Ap8haDNrW4oQOzOhq5+thdmTYOqvLZalkMDtpBU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Ot2r9b5l8X63IhpqRLc+J2hvw7AAYItvYKD1Ap8haDNrW4oQOzOhq5+thdmTYOqvLZalkMDtpBU=',
   'x-amz-request-id': 'E6570B2A04B54027',
   'date': 'Sun, 30 Aug 2020 00:45:36 GMT',
   'etag': '"90f88f1a07fdc88ad2b124948b535dd3"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"90f88f1a07fdc88ad2b124948b535dd3"'}